In [1]:
import pandas as pd
import numpy as np
import os

project_path = r'C:\Users\cneva\06-2024 Instacart Basket Analysis'

### 4.6 Task

#### 3. In a new notebook, import the orders_products_combined dataframe from the pickle file you just saved.

In [3]:
df_orders_products = pd.read_pickle(os.path.join(project_path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

In [5]:
df_prods = pd.read_csv(os.path.join(project_path, '02 Data', 'Prepared Data', 'products_clean.csv'), index_col = 0)

#### 4. Check the shape of the imported dataframe (it should be the same as the one you exported—always check!).

In [7]:
df_orders_products.shape

(32434489, 11)

#### 5. Determine a suitable way to combine the orders_products_combined dataframe with your products data set. Make sure you’re using your wrangled, cleaned, and deduped products data set stored in your “Prepared Data” folder from the previous Exercise’s task.

In [17]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [19]:
df_orders_products.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,True,196,1,0
1,2539329,1,1,2,8,NaN,True,14084,2,0
2,2539329,1,1,2,8,NaN,True,12427,3,0
3,2539329,1,1,2,8,NaN,True,26088,4,0
4,2539329,1,1,2,8,NaN,True,26405,5,0


In [13]:
df_orders_products = df_orders_products.drop(columns = ['_merge'])

In [15]:
df_ords_prods_merged = df_orders_products.merge(df_prods, on = 'product_id', indicator = True, how = 'outer')

#### 6. Confirm the results of the merge using the merge flag.

In [23]:
df_ords_prods_merged['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

In [39]:
 df_ords_prods_merged[df_ords_prods_merged['_merge'].isin(['left_only'])].head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
11552,3331846.0,28.0,3.0,0.0,16.0,2.0,False,34,6.0,0.0,NaN,NaN,NaN,NaN,left_only
11553,1385910.0,28.0,21.0,2.0,17.0,1.0,False,34,1.0,1.0,NaN,NaN,NaN,NaN,left_only
11554,2661186.0,183.0,3.0,0.0,19.0,9.0,False,34,8.0,0.0,NaN,NaN,NaN,NaN,left_only


In [37]:
 df_ords_prods_merged[df_ords_prods_merged['_merge'].isin(['right_only'])].head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
1738025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3630,NaN,NaN,Protein Granola Apple Crisp,57.0,14.0,2.6,right_only
1768779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3718,NaN,NaN,Wasabi Cheddar Spreadable Cheese,21.0,16.0,12.0,right_only
4043097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7045,NaN,NaN,Unpeeled Apricot Halves in Heavy Syrup,88.0,13.0,6.8,right_only


##### There are 30,200 rows that are missing in the prods tables.  These are probably the products that were missing values and were dropped when cleaning the products table.
##### There are 11 rows that are missing from the ords_prods_prior_merge table.  These are probably products that have never been purchased
##### Because of the relatively low mismatches and we know the source/reason for the missing data an inner join will suffice.

In [43]:
df_ords_prods_merged = df_orders_products.merge(df_prods, on = 'product_id', indicator = True, how = 'inner')

In [45]:
df_ords_prods_merged.shape

(32404859, 15)

In [49]:
df_ords_prods_merged.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both


In [51]:
df_ords_prods_merged['_merge'].value_counts(dropna = False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [53]:
df_ords_prods_merged = df_ords_prods_merged.drop(columns = ['_merge'])

#### 7. Export the newly created dataframe as ords_prods_merge in a suitable format (taking into consideration the size).

In [55]:
df_ords_prods_merged.to_pickle(os.path.join(project_path, '02 Data', 'Prepared Data', 'orders_and_products_merged_2.pkl'))

#### 8. Ensure your notebooks and Instacart project folder are organized and that comments and section headings have been used throughout your code. All your exported data files should be effectively labeled and stored in your “Data” folder.

#### 9. Save the two notebooks and send them to your tutor along with a screenshot of the exported data sets in your Instacart project folder.